In [1]:
import pandas as pd
import json
import numpy as np
import os
from pathlib import Path
from ast import literal_eval
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier


In [4]:
data = pd.read_json('data-old/replay_summaries.json')
data.dropna(inplace=True)

In [5]:
def parse_and_count_units(unit_str):
    # Convert the string representation of the list into an actual list
    # units_list = literal_eval(unit_str)
    # Count the occurrence of each unit type
    unit_counts = Counter(unit_str)
    return unit_counts

In [13]:
player_1_units_counts = data['player_1_units'].apply(parse_and_count_units)
player_2_units_counts = data['player_2_units'].apply(parse_and_count_units)
# Combine all unit counts to identify all unique units in the dataset
all_unit_counts = pd.concat([player_1_units_counts, player_2_units_counts])
all_unique_units = set(unit for counts in all_unit_counts for unit in counts)

player_1_cols = [f'player_1_{unit}' for unit in all_unique_units]
player_2_cols = [f'player_2_{unit}' for unit in all_unique_units]

In [14]:
new_cols_df = pd.DataFrame(0, index=data.index, columns=player_1_cols + player_2_cols)


In [18]:
all_unique_units

{'Adept',
 'Archon',
 'AutoTurret',
 'Baneling',
 'Banshee',
 'Battlecruiser',
 'Broodling',
 'BroodlingEscort',
 'Carrier',
 'Changeling',
 'ChangelingMarine',
 'ChangelingMarineShield',
 'ChangelingZealot',
 'ChangelingZergling',
 'ChangelingZerglingWings',
 'Colossus',
 'CommandCenter',
 'Corruptor',
 'Cyclone',
 'Disruptor',
 'Drone',
 'Ghost',
 'GhostAlternate',
 'Hatchery',
 'Hellion',
 'HellionTank',
 'Hydralisk',
 'Immortal',
 'Infestor',
 'Interceptor',
 'KD8Charge',
 'Larva',
 'Liberator',
 'LocustMP',
 'LocustMPPrecursor',
 'MULE',
 'Marauder',
 'Marine',
 'Medivac',
 'Mothership',
 'Mutalisk',
 'Nexus',
 'Nuke',
 'Observer',
 'Oracle',
 'Overlord',
 'ParasiticBombDummy',
 'ParasiticBombRelayDummy',
 'Phoenix',
 'Probe',
 'Queen',
 'Raven',
 'Reaper',
 'Roach',
 'SCV',
 'Sentry',
 'SiegeTank',
 'Stalker',
 'SwarmHostMP',
 'Tempest',
 'Thor',
 'Ultralisk',
 'VikingFighter',
 'Viper',
 'VoidRay',
 'WarpPrism',
 'WidowMine',
 'Zealot',
 'Zergling'}